In [37]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


In [38]:
# загружаем файлы и преобразуем даты
df_customer = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-antonan-antonov/files_for_first_project/olist_customers_dataset.csv')

df_orders = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-antonan-antonov/files_for_first_project/olist_orders_dataset.csv', \
                       parse_dates = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'])

df_items = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-antonan-antonov/files_for_first_project/olist_order_items_dataset.csv', \
                      parse_dates = ['shipping_limit_date'])

In [39]:
# объединяем данные в один датафрэйм
df = df_customer.merge(df_orders, how= 'inner', on = 'customer_id').merge(df_items, how = 'inner', on = 'order_id')

In [41]:
# 1. Сколько у нас пользователей, которые совершили покупку только один раз?

# Покупкой будем считать заказы, где есть подтверждение оплаты (есть время подтверждения оплаты заказа), 
# статус заказа не является отменённым(canceled) или недоступным(unavailable)

#создадим датафрейм с покупками
df_buy = df.query('order_approved_at.notna() and order_status != "canceled" and order_status != "unavailable"') 

#посчитаем количество пользователей, которые совершили покупку только один раз
customers_with_one_order = df_buy.groupby('customer_unique_id', as_index = False).agg({'order_id': 'nunique'}) \
    .query('order_id == 1')['customer_unique_id'].count()

customers_with_one_order
# Вывод: пользователей совершивших всего одну покупку 92084

92084

In [42]:
# 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

# выведем заказы не имеющие дату доставки, статус отличный от "доставлен"(delivered) и запишем в df_not_delivered
df_not_delivered = df.query('order_delivered_customer_date.isna() and order_status != "delivered"')

In [48]:
# создаем колонку с годом и месяцем обещанной даты заказа
df_not_delivered['order_estimated_delivery_month'] = df_not_delivered.order_estimated_delivery_date.dt.to_period('M')


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
# Посмотрим на количество недоставленных заказов по статусам и месяцам
df_not_delivered.groupby(['order_estimated_delivery_month', 'order_status'], as_index=False) \
    .agg({'order_id':'nunique'}) \
    .rename(columns={'order_id':'count_orders'})

,order_estimated_delivery_month,order_status,count_orders
0,2016-10,canceled,2
1,2016-10,shipped,1
2,2016-11,canceled,1
3,2016-11,invoiced,7
4,2016-11,shipped,1
...,...,...,...
84,2018-08,canceled,60
85,2018-08,invoiced,34
86,2018-08,processing,1
87,2018-08,shipped,90


In [51]:
# заказы со статусом отличным от unavailable и canceled объединим в причину "товар еще не был отправлен покупателю"(not_sent)
# и расчитаем количество таких заказов для каждого месяца
not_sent = df_not_delivered.groupby(['order_estimated_delivery_month', 'order_status'], as_index=False) \
    .agg({'order_id':'nunique'}) \
    .rename(columns={'order_id':'count_orders'}) \
    .query('order_status != "canceled" and order_status != "unavailable"') \
    .groupby('order_estimated_delivery_month', as_index=False).agg({'count_orders':'sum'})

# Создаю колонку с названием причины для будущей склейки
not_sent['order_status'] = 'not_sent'

# Определяю порядок столбцов
not_sent = not_sent[['order_estimated_delivery_month', 'order_status', 'count_orders']]

not_sent.head()

,order_estimated_delivery_month,order_status,count_orders
0,2016-10,not_sent,1
1,2016-11,not_sent,8
2,2016-12,not_sent,17
3,2017-02,not_sent,8
4,2017-03,not_sent,101


In [52]:
# заказы со статусом unavailable и canceled являются причиной недоступности и отмены соответственно
# также расчитаем количество таких заказов для каждого месяц
unavailable_or_canceled = df_not_delivered.groupby(['order_estimated_delivery_month', 'order_status'], as_index=False) \
    .agg({'order_id':'nunique'}) \
    .rename(columns={'order_id':'count_orders'}) \
    .query('order_status == "canceled" or order_status == "unavailable"')

unavailable_or_canceled.head()

,order_estimated_delivery_month,order_status,count_orders
0,2016-10,canceled,2
2,2016-11,canceled,1
5,2016-11,unavailable,4
6,2016-12,canceled,5
10,2016-12,unavailable,2


In [53]:
# Объеденим таблицы с причинами(колонку order_status переименуем в cause) недоставленных заказов.
couses_not_delivered = not_sent.append(unavailable_or_canceled) \
    .sort_values('order_estimated_delivery_month') \
    .reset_index() \
    .drop(columns={'index'}) \
    .rename(columns={'order_status': 'cause'})

couses_not_delivered.head()

,order_estimated_delivery_month,cause,count_orders
0,2016-10,not_sent,1
1,2016-10,canceled,2
2,2016-11,canceled,1
3,2016-11,unavailable,4
4,2016-11,not_sent,8


In [54]:
# выводим среднее количесство недоставленных заказов в месяц по разным причинам.
couses_not_delivered.groupby('cause').agg({'count_orders': 'mean'})

# Вывод: в среднем больше всего заказов не доставляется по причине not_sent ('еще не отправлен')

,count_orders
cause,
canceled,20.681818
not_sent,74.826087
unavailable,3.000000


In [12]:
# 3.По каждому товару определить, в какой день недели товар чаще всего покупается.

In [55]:
# создадим таблицу с датой создания заказа, идентификатором заказа, 
# идентификатором товара из датафрейма с покупками (df_buy с 1 шага) и созадим колонку с названием дня недели.

df3 = df_buy[['order_purchase_timestamp', 'order_id', 'product_id']]
df3['weekday'] = df_buy.order_purchase_timestamp.dt.day_name()

df3.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,order_purchase_timestamp,order_id,product_id,weekday
0,2017-05-16 15:05:35,00e7ee1b050b8499577073aeb2a297a1,a9516a079e37a9c9c36b9b78b10169e8,Tuesday
1,2018-01-12 20:48:24,29150127e6685892b6eab3eec79f59c7,4aa6014eceb682077f9dc4bffebc05b0,Friday
2,2018-05-19 16:07:45,b2059ed67ce144a36e2aa97d2c9e9ad2,bd07b66896d6f1494f5b86251848ced7,Saturday
3,2018-03-13 16:06:38,951670f92359f4fe4a63112aa7306eba,a5647c44af977b148e0a3a4751a09e2e,Tuesday
4,2018-07-29 09:51:30,6b7d50bd145f6fc7f33cebabd7e49d0f,9391a573abe00141c56e38d84d7d5b3b,Sunday


In [67]:
# для каждого товара выведем день недели в который он чаще всего покупался
weekday_max_orders =df3.groupby(['product_id', 'weekday'],as_index=False) \
    .agg({'order_id':'count'}) \
    .groupby(['product_id'])[['weekday']].max().sort_values('product_id')

weekday_max_orders.head()

,weekday
product_id,
00066f42aeeb9f3007548bb9d3f33c38,Sunday
00088930e925c41fd95ebfe695fd2655,Tuesday
0009406fd7479715e4bef61dd91f2462,Thursday
000b8f95fcb9e0096488278317764d19,Wednesday
000d9be29b5207b54e86aa1b1ac54872,Tuesday


In [68]:
# 4.Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? 
# Не стоит забывать, что внутри месяца может быть не целое количество недель. 
# Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. 

In [69]:
# Выведем  из датафрема с покупками (df_buy) уникальный идентификатор пользователя, 
# время создания заказа и идентификатор заказа
df4 = df_buy[['customer_unique_id', 'order_purchase_timestamp', 'order_id']]

# создаем колонку с годом и месяцем создания заказа
df4['month'] = df4.order_purchase_timestamp.dt.to_period('M')

# создаем колонку с количеством недель в месяце (округлил до сотых)
df4['count_week'] = (df4.month.dt.day / 7).round(2)

df4.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,customer_unique_id,order_purchase_timestamp,order_id,month,count_week
0,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35,00e7ee1b050b8499577073aeb2a297a1,2017-05,4.43
1,290c77bc529b7ac935b93aa66c333dc3,2018-01-12 20:48:24,29150127e6685892b6eab3eec79f59c7,2018-01,4.43
2,060e732b5b29e8181a18229c7b0b2b5e,2018-05-19 16:07:45,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05,4.43
3,259dac757896d24d7702b9acbbff3f3c,2018-03-13 16:06:38,951670f92359f4fe4a63112aa7306eba,2018-03,4.43
4,345ecd01c38d18a9036ed96c73b8d066,2018-07-29 09:51:30,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07,4.43


In [70]:
# расчитываю количество покупок совершенных в месяц для каждого пользователя
df4_avg_bay = df4.groupby(['month', 'customer_unique_id', 'count_week'], as_index = False).agg({'order_id': 'count'})

df4_avg_bay.head()

,month,customer_unique_id,count_week,order_id
0,2016-09,830d5b7aaa3b6f1e9ad63703bec97d23,4.29,3
1,2016-09,b7d76e111c89f7ebf14761390f0f7d17,4.29,2
2,2016-10,0032c76b20340da25249092a268ce66c,4.43,1
3,2016-10,01f156677184504063bd19739f924af1,4.43,1
4,2016-10,0313291a6f4f16df04dcf819d88c38ef,4.43,1


In [72]:
# Создаю колонку (avg_bay_to_week) со средним количеством покупок в неделю для каждого месяца
# (делю количество покупок на количество недель в месяце), округлил до сотых.
df4_avg_bay['avg_bay_to_week'] = (df4_avg_bay.order_id / df4_avg_bay.count_week).round(2)

#  Вывожу среднее количество покупок в неделю (по месяцам) каждого из пользователей  
df4_avg_bay[['month', 'customer_unique_id', 'avg_bay_to_week']].head()

,month,customer_unique_id,avg_bay_to_week
0,2016-09,830d5b7aaa3b6f1e9ad63703bec97d23,0.70
1,2016-09,b7d76e111c89f7ebf14761390f0f7d17,0.47
2,2016-10,0032c76b20340da25249092a268ce66c,0.23
3,2016-10,01f156677184504063bd19739f924af1,0.23
4,2016-10,0313291a6f4f16df04dcf819d88c38ef,0.23


In [73]:
# 5. Используя pandas, проведи когортный анализ пользователей. 
# В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.

In [74]:
# определим кагорты пользователей по штату доставки (customer_state)
# период с января по декабрь возьмем за 2017 год, так как явлется единственным полным годом

# Выведем  из датафрейма (df) штат доставки, уникальный идентификатор пользователя, 
# время создания заказа
df5 = df[['customer_state', 'customer_unique_id', 'order_purchase_timestamp']]

# создаю колонку с годом и месяцем создания заказа
df5['month'] = df5['order_purchase_timestamp'].dt.to_period('M')

# расчитал количество уникальных пользователей по штатам на начало 2017 года
users_2016 = df5.query('"2016-12">= month').groupby('customer_state', as_index= False) \
    .agg({'customer_unique_id': 'nunique'})

# создал колонку для склейки с будующи датафреймом 2017 года
users_2016['month'] = pd.to_datetime('2016-12').to_period('M') 

users_2016.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,customer_state,customer_unique_id,month
0,AL,2,2016-12
1,BA,4,2016-12
2,CE,7,2016-12
3,DF,6,2016-12
4,ES,4,2016-12


In [75]:
# отсортировал необходимый период(2017г.) и расчитал количество уникальных 
# пользователей совершавших заказы в каждом штате по месяцам. Дополнил данными на начало 2017 года(из users_2016)
df5_2017_users = df5.query('"2017-12">= month >= "2017-01"').sort_values('month') \
    .groupby(['month', 'customer_state'],as_index = False).agg({'customer_unique_id':'nunique'}) \
    .append(users_2016) \
    .pivot(index='customer_state', columns='month', values = 'customer_unique_id').fillna(0)

df5_2017_users.head()

month,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
customer_state,,,,,,,,,,,,,
AC,0.0,1.0,3.0,2.0,5.0,8.0,4.0,5.0,4.0,5.0,6.0,5.0,5.0
AL,2.0,2.0,12.0,10.0,22.0,25.0,10.0,16.0,18.0,20.0,28.0,26.0,12.0
AM,0.0,0.0,5.0,5.0,13.0,9.0,1.0,5.0,5.0,9.0,3.0,10.0,6.0
AP,0.0,0.0,2.0,3.0,0.0,5.0,2.0,1.0,3.0,2.0,3.0,4.0,4.0
BA,4.0,24.0,58.0,90.0,92.0,122.0,102.0,151.0,154.0,167.0,162.0,237.0,189.0


In [76]:
# расчитываю Retention. 
# делю число активных пользователей (за период 3 месяца) на общее количество пользователей к текущему месяцу и умножаю на 100.
# округлил до сотых.
df5_2017_retention = ((df5_2017_users.rolling(3, axis=1).sum() \
                       / df5_2017_users.rolling(100, min_periods = 1,axis = 1)\
                       .sum())*100).round(2)

df5_2017_retention.head()

month,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
customer_state,,,,,,,,,,,,,
AC,NaN,NaN,100.0,100.00,90.91,78.95,73.91,60.71,40.62,37.84,34.88,33.33,30.19
AL,NaN,NaN,100.0,92.31,91.67,78.08,68.67,51.52,37.61,39.42,40.00,38.74,32.51
AM,NaN,NaN,100.0,100.00,100.00,84.38,69.70,39.47,25.58,36.54,30.91,33.85,26.76
AP,NaN,NaN,100.0,100.00,100.00,80.00,58.33,61.54,37.50,33.33,38.10,36.00,37.93
BA,NaN,NaN,100.0,97.73,89.55,77.95,64.23,58.32,51.07,48.96,42.90,41.53,37.89


In [77]:
# создал колонку со средним значением Retention, округляю до сотых
df5_2017_retention['avg_retention'] =df5_2017_retention.mean(axis=1).round(2)

# вывожу штат(когорту) с самым высоким значением.
df5_2017_retention[['avg_retention']].sort_values('avg_retention', ascending=False).head(1)

# Вывод: когорта с самым высоким retention на 3й месяц - MS

month,avg_retention
customer_state,
MS,66.35


In [78]:
# 6.Часто для качественного анализа аудитории использую подходы, основанные на сегментации. 
# Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. 
# В кластеризации можешь выбрать следующие метрики: 
# R - время от последней покупки пользователя до текущей даты, 
# F - суммарное количество покупок у пользователя за всё время, 
# M - сумма покупок за всё время. 
# Подробно опиши, как ты создавал кластеры. 
# Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. 
# Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, 
# frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. 

In [79]:
# выведем количество товаров в каждом заказе, что бы понять, что каждый заказ может включать несколько товаров
df_buy.groupby(['order_id']).agg({'product_id':'count'}).sort_values('product_id', ascending=False).head()

,product_id
order_id,
8272b63d03f5f79c56e9e4120aec44ef,21
ab14fdcfbe524636d65ee38360e22ce8,20
1b15974a0141d54e36626dca3fdc731a,20
428a2f660dc84138d969ccd69a0ab6d5,15
9ef13efd6949e4573a18964dd1bbe7f5,15


In [80]:
# Создаю временный фрейм данных с идентификатором пользователя, временем создания заказа, идентификатором заказа,
# ид товара, ценой за единицу товара и количиством продуктов в заказе
df6_first = df_buy.groupby(['customer_unique_id', 'order_purchase_timestamp', 'order_id', 'product_id', 'price'], as_index = False) \
    .agg(num_products= ('product_id','count')).sort_values('order_purchase_timestamp').reset_index(drop=True)

# Создаю колонку со стоимостью определенных товаров в заказе исходя из количества
df6_first['sum_price']= df6_first.price * df6_first.num_products

df6_first.head()

,customer_unique_id,order_purchase_timestamp,order_id,product_id,price,num_products,sum_price
0,b7d76e111c89f7ebf14761390f0f7d17,2016-09-04 21:15:19,2e7a8482f6fb09756ca50c10d7bfc047,c1488892604e4ba5cff5b4eb4d595400,39.99,1,39.99
1,b7d76e111c89f7ebf14761390f0f7d17,2016-09-04 21:15:19,2e7a8482f6fb09756ca50c10d7bfc047,f293394c72c9b5fafd7023301fc21fc2,32.90,1,32.90
2,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,44.99,3,134.97
3,32ea3bdedab835c3aa6cb68ce66565ef,2016-10-03 09:44:50,3b697a20d9e427646d92567910af6d57,3ae08df6bcbfe23586dd431c40bddbb7,29.90,1,29.90
4,2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,be5bc2f0da14d8071e2d45451ad119d9,fd7fd78fd3cbc1b0a6370a7909c0a629,21.90,1,21.90


In [81]:
# Считаю стоимость заказа (order_price) и записываю в df6
df6 = df6_first.groupby(['order_id', 'order_purchase_timestamp', 'customer_unique_id'], as_index = False) \
    .agg(order_price = ('sum_price','sum')).sort_values('order_purchase_timestamp').reset_index(drop=True)

df6.head()

,order_id,order_purchase_timestamp,customer_unique_id,order_price
0,2e7a8482f6fb09756ca50c10d7bfc047,2016-09-04 21:15:19,b7d76e111c89f7ebf14761390f0f7d17,72.89
1,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,830d5b7aaa3b6f1e9ad63703bec97d23,134.97
2,3b697a20d9e427646d92567910af6d57,2016-10-03 09:44:50,32ea3bdedab835c3aa6cb68ce66565ef,29.90
3,be5bc2f0da14d8071e2d45451ad119d9,2016-10-03 16:56:50,2f64e403852e6893ae37485d5fcacdaf,21.90
4,a41c8759fbe7aab36ea07e038b2d4465,2016-10-03 21:13:36,61db744d2f835035a5625b59350c6b63,36.49


In [82]:
# В качестве текущей даты установим дату на один день позже последней даты заказа
now_date = df6['order_purchase_timestamp'].max() + timedelta(days=1)
now_date

Timestamp('2018-09-04 09:06:57')

In [83]:
# добавляю столбец с количеством дней между покупкой и условной текущей датой(now)
df6['days_since_order'] = df6['order_purchase_timestamp'].apply(lambda x: (now_date - x).days)
df6.head()

,order_id,order_purchase_timestamp,customer_unique_id,order_price,days_since_order
0,2e7a8482f6fb09756ca50c10d7bfc047,2016-09-04 21:15:19,b7d76e111c89f7ebf14761390f0f7d17,72.89,729
1,bfbd0f9bdef84302105ad712db648a6c,2016-09-15 12:16:38,830d5b7aaa3b6f1e9ad63703bec97d23,134.97,718
2,3b697a20d9e427646d92567910af6d57,2016-10-03 09:44:50,32ea3bdedab835c3aa6cb68ce66565ef,29.90,700
3,be5bc2f0da14d8071e2d45451ad119d9,2016-10-03 16:56:50,2f64e403852e6893ae37485d5fcacdaf,21.90,700
4,a41c8759fbe7aab36ea07e038b2d4465,2016-10-03 21:13:36,61db744d2f835035a5625b59350c6b63,36.49,700


In [84]:
# Для каждого пользвотеля расчитываю: 
# количество дней с момента последнего заказа (Recency),
# общее количество заказов за все время(Frequency) и
# сумму покупок за всё время (Monetary).
rfm = df6.groupby('customer_unique_id') \
    .agg(Recency = ('days_since_order', 'min'), \
         Frequency = ('order_id', 'count'), \
         Monetary = ('order_price','sum')) \
         .reset_index()
rfm.head()

,customer_unique_id,Recency,Frequency,Monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90
2,0000f46a3911fa3c0805444483337064,542,1,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00


In [85]:
# границы метрик буду определять по категориям с помощью cut.
# для Recency недавним значением буду считать дату последнего заказа в последние 90 дней, давним - более 365 дней
# для Frequency лучшим значением буду считать более 10 заказов, худшим - менее 2 заказов
# для Monetary лучшим значением буду считать сумму покупок более чем на 3 000, худшим - на 500 и менее
rfm['R'] = pd.cut(rfm['Recency'], [0, 90, 365, 800], labels=['3','2','1'])
rfm['F'] = pd.cut(rfm['Frequency'], [0, 1, 10, 20], labels=['1','2','3'])
rfm['M'] = pd.cut(rfm['Monetary'], [0, 500, 3000, 15000], labels=['1','2','3'])

rfm.head()

,customer_unique_id,Recency,Frequency,Monetary,R,F,M
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90,2,1,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90,2,1,1
2,0000f46a3911fa3c0805444483337064,542,1,69.00,1,1,1
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99,2,1,1
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00,2,1,1


In [86]:
# объединяю полученные категории в RFM
rfm['RFM'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

rfm.head()

,customer_unique_id,Recency,Frequency,Monetary,R,F,M,RFM
0,0000366f3b9a7992bf8c76cfdf3221e2,116,1,129.90,2,1,1,211
1,0000b849f77a49e4a4ce2b2a4ca5be3f,119,1,18.90,2,1,1,211
2,0000f46a3911fa3c0805444483337064,542,1,69.00,1,1,1,111
3,0000f6ccb0745a6a4b88665a16c9f078,326,1,25.99,2,1,1,211
4,0004aac84e0df4da2b147fca70cf8255,293,1,180.00,2,1,1,211


In [87]:
# определим сегменты
rfm['Segment'] =  \
    rfm['R'].replace({'1':'Давние', '2':'Спящие', '3':'Недавние'}) + '/' + \
    rfm['F'].replace({'1':'Разовые', '2':'Редкие', '3':'Частые'}) + '/' + \
    rfm['M'].replace({'1':'Низкий чек', '2':'Средний чек', '3':'Высокий чек'})

In [88]:
# Вывожу количество Клиентов по заданным сегментам.
rfm.groupby(['RFM','Segment']).agg(customers_count=('customer_unique_id','count')) \
    .sort_values('customers_count', ascending=False).reset_index()

# Вывод: больше всего клиентов совершили разовые покупки в промежутке от 91 до 365 дней назад, при этом они имеют низкий чек.

,RFM,Segment,customers_count
0,211,Спящие/Разовые/Низкий чек,51481
1,111,Давние/Разовые/Низкий чек,20552
2,311,Недавние/Разовые/Низкий чек,16624
3,212,Спящие/Разовые/Средний чек,1994
4,221,Спящие/Редкие/Низкий чек,1567
5,112,Давние/Разовые/Средний чек,791
6,312,Недавние/Разовые/Средний чек,610
7,321,Недавние/Редкие/Низкий чек,533
8,121,Давние/Редкие/Низкий чек,498
9,222,Спящие/Редкие/Средний чек,179
